# Align the Antocyanin QTL for all four haplotypes of the baby blueberry progeny of M7 and Nui.

In [39]:
WKDIR=/workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning
LOGDIR=/workspace/hraijc/Blueberry/Blueberry_trio/log


VCAE_TARGETS=${WKDIR}/AnthoQTL_Vcae.fasta
VCAE_REGION=${WKDIR}/VcaeV1.3.p0.region8-9mb.fa
HAPLOTYPE_DIR=/workspace/hraijc/Blueberry/Blueberry_trio/phased_QC

In [37]:
cd $WKDIR

In [16]:
cd $WKDIR
# Symlink Chen's assemblies into my workspace.
for FILE in /workspace/hraczw/github/bioinf_blueberry_trio/*.fa
    do
    ln -s $FILE
done

In [27]:
#For loops are not working in slurm. Defaulting to copy and paste output from here to next cell.
for HAP2 in *ctg.fa
do
    echo makeblastdb -in ${HAP2} -parse_seqids -dbtype nucl
    echo blastn -task dc-megablast -db ${HAP2} -query ${VCAE_TARGETS} -outfmt 6 -evalue 1e-50 -num_threads 8 -out ${HAP2%.fa}_blast_out.txt
    #echo ${HAP2}
done

makeblastdb -in M7.bp.hap1.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db M7.bp.hap1.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_Vcae.fasta -outfmt 6 -evalue 1e-50 -num_threads 8 -out M7.bp.hap1.p_ctg_blast_out.txt
makeblastdb -in M7.bp.hap2.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db M7.bp.hap2.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_Vcae.fasta -outfmt 6 -evalue 1e-50 -num_threads 8 -out M7.bp.hap2.p_ctg_blast_out.txt
makeblastdb -in Nui.bp.hap1.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db Nui.bp.hap1.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_Vcae.fasta -outfmt 6 -evalue 1e-50 -num_threads 8 -out Nui.bp.hap1.p_ctg_blast_out.txt
makeblastdb -in Nui.bp.hap2.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db Nui.bp.hap2.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_V

In [28]:
# Blast haplotyped assemblies to the Vcae Ant QTL 
cd $WKDIR

sbatch << EOF
#!/bin/bash
#SBATCH -J hic_compqc
#SBATCH -o ${LOGDIR}/%J.out
#SBATCH -e ${LOGDIR}/%J.err
#SBATCH --cpus-per-task=8
#SBATCH --mem=1G
#SBATCH --time=04:10:00

module load ncbi-blast/2.11.0 


makeblastdb -in M7.bp.hap1.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db M7.bp.hap1.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_Vcae.fasta -outfmt 6 -evalue 1e-50 -num_threads 8 -out M7.bp.hap1.p_ctg_blast_out.txt
makeblastdb -in M7.bp.hap2.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db M7.bp.hap2.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_Vcae.fasta -outfmt 6 -evalue 1e-50 -num_threads 8 -out M7.bp.hap2.p_ctg_blast_out.txt
makeblastdb -in Nui.bp.hap1.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db Nui.bp.hap1.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_Vcae.fasta -outfmt 6 -evalue 1e-50 -num_threads 8 -out Nui.bp.hap1.p_ctg_blast_out.txt
makeblastdb -in Nui.bp.hap2.p_ctg.fa -parse_seqids -dbtype nucl
blastn -task dc-megablast -db Nui.bp.hap2.p_ctg.fa -query /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/AnthoQTL_Vcae.fasta -outfmt 6 -evalue 1e-50 -num_threads 8 -out Nui.bp.hap2.p_ctg_blast_out.txt

EOF

Submitted batch job 1040708


In [40]:
cd $WKDIR
for HAP in *ctg.fa
do
    echo "cut -f2 ${HAP%.fa}_blast_out.txt | sort | uniq > ${HAP%.fa}_hits.txt"
    echo "seqtk subseq ${HAP} ${HAP%.fa}_hits.txt > ${HAP%.fa}_hits.fa"
    echo "minimap2 -t 4 -cx asm10 ${HAP%.fa}_hits.fa ${VCAE_REGION} > ${HAP%.fa}_AnthQTL_asm10.paf"     
done

cut -f2 M7.bp.hap1.p_ctg_blast_out.txt | sort | uniq > M7.bp.hap1.p_ctg_hits.txt
seqtk subseq M7.bp.hap1.p_ctg.fa M7.bp.hap1.p_ctg_hits.txt > M7.bp.hap1.p_ctg_hits.fa
minimap2 -t 4 -cx asm10 M7.bp.hap1.p_ctg_hits.fa /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa > M7.bp.hap1.p_ctg_AnthQTL_asm10.paf
cut -f2 M7.bp.hap2.p_ctg_blast_out.txt | sort | uniq > M7.bp.hap2.p_ctg_hits.txt
seqtk subseq M7.bp.hap2.p_ctg.fa M7.bp.hap2.p_ctg_hits.txt > M7.bp.hap2.p_ctg_hits.fa
minimap2 -t 4 -cx asm10 M7.bp.hap2.p_ctg_hits.fa /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa > M7.bp.hap2.p_ctg_AnthQTL_asm10.paf
cut -f2 Nui.bp.hap1.p_ctg_blast_out.txt | sort | uniq > Nui.bp.hap1.p_ctg_hits.txt
seqtk subseq Nui.bp.hap1.p_ctg.fa Nui.bp.hap1.p_ctg_hits.txt > Nui.bp.hap1.p_ctg_hits.fa
minimap2 -t 4 -cx asm10 Nui.bp.hap1.p_ctg_hits.fa /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa > Nui.bp.hap1.p_ctg_AnthQT

In [46]:
# Extract contigs with hits. then map
cd $WKDIR
sbatch << EOF
#!/bin/bash
#SBATCH -J minimap
#SBATCH -o ${LOGDIR}/%J.out
#SBATCH -e ${LOGDIR}/%J.err
#SBATCH --cpus-per-task=4
#SBATCH --mem=12G
#SBATCH --time=01:10:00

module load minimap2
module load seqtk

#cut -f2 M7.bp.hap1.p_ctg_blast_out.txt | sort | uniq > M7.bp.hap1.p_ctg_hits.txt
#seqtk subseq M7.bp.hap1.p_ctg.fa M7.bp.hap1.p_ctg_hits.txt > M7.bp.hap1.p_ctg_hits.fa
#minimap2 -t 4 -cx asm10 M7.bp.hap1.p_ctg_hits.fa /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa > M7.bp.hap1.p_ctg_AnthQTL_asm10.paf
#cut -f2 M7.bp.hap2.p_ctg_blast_out.txt | sort | uniq > M7.bp.hap2.p_ctg_hits.txt
#seqtk subseq M7.bp.hap2.p_ctg.fa M7.bp.hap2.p_ctg_hits.txt > M7.bp.hap2.p_ctg_hits.fa
#minimap2 -t 4 -cx asm10 M7.bp.hap2.p_ctg_hits.fa /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa > M7.bp.hap2.p_ctg_AnthQTL_asm10.paf
#cut -f2 Nui.bp.hap1.p_ctg_blast_out.txt | sort | uniq > Nui.bp.hap1.p_ctg_hits.txt
seqtk subseq Nui.bp.hap1.p_ctg.fa Nui.bp.hap1.p_ctg_hits.txt > Nui.bp.hap1.p_ctg_hits.fa
#minimap2 -t 4 -cx asm10 Nui.bp.hap1.p_ctg_hits.fa /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa > Nui.bp.hap1.p_ctg_AnthQTL_asm10.paf
#cut -f2 Nui.bp.hap2.p_ctg_blast_out.txt | sort | uniq > Nui.bp.hap2.p_ctg_hits.txt
seqtk subseq Nui.bp.hap2.p_ctg.fa Nui.bp.hap2.p_ctg_hits.txt > Nui.bp.hap2.p_ctg_hits.fa
#minimap2 -t 4 -cx asm10 Nui.bp.hap2.p_ctg_hits.fa /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa > Nui.bp.hap2.p_ctg_AnthQTL_asm10.paf
EOF

Submitted batch job 1040872


In [43]:
du -sh /workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa

du: cannot access ‘/workspace/hraijc/Blueberry/Blueberry_trio/QTL_Minning/VcaeV1.3.p0.region8-9mb.fa’: No such file or directory


: 1

In [33]:
module load minimap2
minimap2 -t 4 -cx asm10 AnthoQTL_Vcae.fasta VcaeV1.3.p0.regions.fa > QTL_AnthQTL_asm10.paf


[M::mm_idx_gen::0.003*0.88] collected minimizers
[M::mm_idx_gen::0.004*1.49] sorted minimizers
[M::main::0.004*1.49] loaded/built the index for 68 target sequence(s)
[M::mm_mapopt_update::0.004*1.48] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 68
[M::mm_idx_stat::0.004*1.47] distinct minimizers: 838 (69.93% are singletons); average occurrences: 1.414; average spacing: 11.477; total length: 13600
[M::worker_pipeline::0.223*1.49] mapped 2 sequences
[M::main] Version: 2.22-r1101
[M::main] CMD: minimap2 -t 4 -cx asm10 AnthoQTL_Vcae.fasta VcaeV1.3.p0.regions.fa
[M::main] Real time: 0.224 sec; CPU: 0.333 sec; Peak RSS: 0.066 GB
